In [ ]:
# Notebook to present comparison (i.e., "diff") of data for two scenarios for a given set of highway links in map form
# using the hvplot library. The "diff" of total flow, speed, and  V/C, are presented.
# The two sets of input data were previously calculated by sample_highway_links_report.ipypnb, and saved in CSV format.

import numpy as np
import pandas as pd
import geopandas as gp
from io import StringIO
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
# >>> *** USER INPUT REQUIRED *** <<<
#
# (1) Directory in which user's output CSV report data was saved - it will now be our *input* directory.
#
my_sandbox_dir = r'S:/my_modx_output_dir/'
#
# (2) Name of CSV file in this directory with report for "base" scenario.
#
base_csv_fn = 'links_report_base_scenario.csv'
#
# (3) Name of CSV file in this directory with report for "comparison" scenario.
#
comparison_csv_fn = 'links_report_comparison_scenario.csv'

In [ ]:
# Fully-qualified pathnames to CSV files containing "base" and "comparison" reports
#
fq_base_csv_fn = my_sandbox_dir + base_csv_fn
fq_comparison_csv_fn = my_sandbox_dir + comparison_csv_fn

In [ ]:
# Read "base" data into a dataframe
#
base_df = pd.read_csv(fq_base_csv_fn, delimiter=',')
#
# Save the list of links for which the reports were generated
#
links_list = base_df['ID1'].to_list()
#
# Save column names for (possible) later use, after having removed the "ID1" column at index[0]
#
saved_column_names = base_df.columns[1:]
#
# Maybe we don't need to do the following, thanks to the functionality of df.merge()...
# And rename each of the colums with the prefix 'base_'
# base_df.rename(columns=lambda x: 'base_' + x, inplace=True, )

In [ ]:
# Read "comparison" data into a dataframe
comparison_df = pd.read_csv(fq_comparison_csv_fn, delimiter=',')
# Maybe we don't need to do the following, thanks to the functionality of df.merge()...
# And rename each of the colums with the prefix 'comparison_'
# comparison_df.rename(columns=lambda x: 'comparison_' + x, inplace=True, )

In [ ]:
delta_df = pd.merge(left=comparison_df, right=base_df, on="ID1", suffixes=('_comp', '_base'))

In [ ]:
delta_df

In [ ]:
# Calculate the delta (comparision - base) for each metric
#
for column_name in column_names:
    base_column_name = column_name + '_base'
    comp_column_name = column_name + '_comp'
    delta_column_name = column_name + '_delta'
    delta_df[delta_column_name] = delta_df[comp_column_name] - delta_df[base_column_name]
# end_for

In [ ]:
delta_df

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG4326.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Filter the links geodataframe to only the links of interest
filtered_links_gdf = links_gdf[links_gdf['ID'].isin(links_list)] 

In [ ]:
filtered_links_gdf

In [ ]:
# Join the geo-data frame for the links with the "links_data_df", which contains the computed data about these links
join_df = filtered_links_gdf.join(delta_df.set_index("ID1"), on="ID")

In [ ]:
join_df

In [ ]:
# Make a static map of the change in speed during the AM period
join_df.plot("Speed_am_delta", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Change in AM Speed')

In [ ]:
# Make an interactive bar chart of the change in speed for each link in the AM period
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='Speed_am_delta', ylabel='Change in Speed (MPH) AM Period', xformatter="%f", height=1500)

In [ ]:
# Make a static map of the change in volume-to-capacity ratio during the AM period
join_df.plot("VOC_am_delta", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Change in VOC in AM')

In [ ]:
# Make an interactive bar chart of the change in volume-to-capacity ratio for each link in the AM period
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='VOC_am_delta', ylabel='Change in V/C Ratio - AM Period', xformatter="%f", height=1500)

In [ ]:
# Make a static map of the change in total daily flow (volume) ratio during the AM period
join_df.plot("Tot_Flow_daily_delta", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Change in Daily Total Flow (volume)')

In [ ]:
# Make an interactive bar chart of the change in total daily flow (volume) by link
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='Tot_Flow_daily_delta', ylabel='Chagne in Total Daily Volume', xformatter="%f", height=1500)